#Summary Analyses

In [1]:
deleteSessions = true;
deleteUnmodulated = true;
saccadeBaseRatio = [];

ddmType = 'ddmRankMean';  % ddm coherence determined by comparing means of easy vs hard spike rates into RF (and out)
ddmType = 'ddm';  % ddm coherence determined by ranksum test and slopes of means easy vs hard rates into RF (and out)


sortByElectrode = false;
electrode = 'uprobe';
channels = [32];



dataPath = fullfile(projectRoot,'data',projectDate,subject);


% Choose the categories of neurons to compare
ddmEpoch = 'Stim';

classicEpoch = 'presacc';
cancelEpoch = 'presacc_cancel';


sessionRemove = ccm_exclude_sessions(subject);
sessionKeep = ccm_session_by_electrode(subject,projectRoot,projectDate, electrode, channels);



% LOAD ALL RELEVANT DATA AND LISTS

% Load full popuulation of neurons
load(fullfile(dataPath, ['ccm_neuronTypes', addMulti]));
if deleteSessions
    neuronTypes = neuronTypes(~ismember(neuronTypes.sessionID, sessionRemove),:);
end

% Figure out how many modulated units there were (modulation of some kind)
modulatedList   = {'fix','vis','checker','presacc','postsacc'};
nModulated      = 0;
modulatedInd    = zeros(size(neuronTypes, 1));
modTable        = neuronTypes(1,1:4); % Table to concatenate all modulated units
modTable(1,:) = [];
for i = 1 : length(modulatedList)
    
    % Open the edited list version of the units
    c = load(fullfile(dataPath, ['ccm_',modulatedList{i},'_neurons', addMulti]));
    classic = c.neurons;
    if deleteSessions
        classic = classic(~ismember(classic.sessionID, sessionRemove),:);
    end
    if sortByElectrode
        classic = classic(ismember(classic.sessionID, sessionKeep),:);
    end
    
    % Add modulated units to modTable
    [~, ~, iClassic] = setxor(modTable, classic);
    modTable = [modTable; classic(iClassic,:)];
    size(modTable)
    
%    [C, neuronTypesInd,ib] = intersect(neuronTypes(:,1:4),classic);
%   neuronTypesLogic = zeros(size(neuronTypes, 1));
%    neuronTypesLogic(neuronTypesInd) = 1;
%    modulatedInd = modulatedInd | neuronTypesLogic;
end    
nModulated = size(modTable, 1)

    
    
    
%{
if deleteUnmodulated
    modulated = neuronTypes.fix | neuronTypes.vis | neuronTypes.checker | neuronTypes.presacc | neuronTypes.postsacc;
    neuronTypes = neuronTypes(modulated,:);
end
nModulated = size(neuronTypes, 1);

if sortByElectrode
    neuronTypes = neuronTypes(ismember(neuronTypes.sessionID, sessionKeep),:);
end
if ~isempty(saccadeBaseRatio)
    includeInd = neuronTypes.saccadeBaseRatio >= saccadeBaseRatio;
    keepUnit = neuronTypes(includeInd, 1:4);
    neuronTypes = neuronTypes(includeInd,:);
end

nUnit = size(neuronTypes, 1);
clear neuronTypes
%}



% Load Classic neuron classifications
c = load(fullfile(dataPath, ['ccm_',classicEpoch,'_neurons', addMulti]));
classic = c.neurons;
if deleteSessions
    classic = classic(~ismember(classic.sessionID, sessionRemove),:);
end
if sortByElectrode
    classic = classic(ismember(classic.sessionID, sessionKeep),:);
end
if ~isempty(saccadeBaseRatio)
    classic = intersect(classic, keepUnit);
end



% Load ddm neuron classifications
dg = load(fullfile(dataPath, ['ccm_',ddmType, ddmEpoch, '_neurons', addMulti]));
ddm = dg.neurons;
if deleteSessions
    ddm = ddm(~ismember(ddm.sessionID, sessionRemove),:);
end
if sortByElectrode
    ddm = ddm(ismember(ddm.sessionID, sessionKeep),:);
end
if ~isempty(saccadeBaseRatio)
    ddm = intersect(ddm, keepUnit);
end





% Load Cancel Time population 
ca = load(fullfile(dataPath, ['ccm_',cancelEpoch,'_neurons', addMulti]));
cancel = ca.neurons;
if deleteSessions
    cancel = cancel(~ismember(cancel.sessionID, sessionRemove),:);
end
if sortByElectrode
    cancel = cancel(ismember(cancel.sessionID, sessionKeep),:);
end
if ~isempty(saccadeBaseRatio)
    cancel = intersect(cancel, keepUnit);
end



[~, iCla, iDdm] = setxor(classic, ddm);
classicDdm = intersect(classic, ddm);
classicNoDdm = classic(iCla, :);
ddmNoClassic = ddm(iDdm, :);


[~, iCla, iCan] = setxor(classic, cancel);
classicCancel = intersect(classic, cancel);
classicNoCancel = classic(iCla, :);
cancelNoClassic = ddm(iCan, :);


[~, iCla, iBoth] = setxor(cancel, classicDdm);
classicCancelNoDdm = cancel(iCla,:);
classicDdmNoCancel = classicDdm(iBoth,:);
classicDdmCancel = intersect(classicDdm, cancel);

classicNoDdmNoCancel = intersect(classicNoDdm, classicNoCancel);


SyntaxError: invalid syntax (<ipython-input-1-dc7a2022653b>, line 5)

###Print out percentages of the relevant neuron categories

In [ ]:
fprintf('\nTotal Modulated Units:\t%d\n', nModulated)

fprintf('\n%s:\t%d\t%.1f%% of total\n', classicEpoch, size(classic, 1), size(classic, 1)*100/nModulated)

fprintf('DDM-%s:\t%d\t%.1f%% of total\n', ddmEpoch, size(ddm, 1), size(ddm, 1)*100/nModulated)

fprintf('%s and DDM-%s:\t%d\t%.1f%% of total,\t%.1f%% of %s,\t%.1f%% of DDM-%s\n', classicEpoch, ddmEpoch, size(classicDdm, 1), size(classicDdm, 1)*100/nModulated, size(classicDdm, 1)*100/size(classic, 1), classicEpoch, size(classicDdm, 1)*100/size(ddm, 1), ddmEpoch)

fprintf('%s and %s:\t%d\t%.1f%% of total,\t%.1f%% of %s\n', classicEpoch, cancelEpoch, size(classicCancel, 1), size(classicCancel, 1)*100/nModulated, size(classicCancel, 1)*100/size(classic, 1), classicEpoch)

fprintf('%s and %s and DDM-%s:\t%d\t%.1f%% of total,\t%.1f%% of %s,\t%.1f%% of DDM-%s\n', classicEpoch, cancelEpoch, ddmEpoch, size(classicDdmCancel, 1), size(classicDdmCancel, 1)*100/nModulated, size(classicDdmCancel, 1)*100/size(classic, 1), classicEpoch, size(classicDdmCancel, 1)*100/size(ddm, 1), ddmEpoch)


## Make a Venn diagram of the results

In [1]:
A = [size(classic, 1) size(cancel, 1) size(ddm, 1)];
I = [size(classicCancel, 1) size(classicDdm, 1) size(classicDdmCancel, 1) size(classicDdmCancel, 1)];

figureHandle = figure(1)
clf
axis equal, axis off
venn(A,I,'FaceColor',{'r','y','b'},'FaceAlpha',{.5,0.6,0.2},'EdgeColor','black')



filePath = fullfile(projectRoot,'results',projectDate,subject);

if ~isempty(saccadeBaseRatio)
    appendBase = '_saccBaseRatio_';
else
    appendBase = [];
end
if multiUnit
    addMulti = '_multiUnit';
else
    addMulti = [];
end
if normalizeData
    addNorm = '_normalized';
else
    addNorm = [];
end

addDdm = ['_',ddmType,'_'];

fileName = ['venn_',classicEpoch,addDdm,addMulti,addNorm,appendBase,'.eps'];

    print(figureHandle, fullfile(filePath, fileName),'-depsc', '-r300')


SyntaxError: invalid syntax (<ipython-input-1-d9d96494dec9>, line 1)